In [2]:
import pandas as pd

geo = pd.read_csv('olist_geolocation_dataset.csv')
agrup_cep = geo.groupby('geolocation_zip_code_prefix').mean(numeric_only=True).reset_index()
geo_ref = pd.DataFrame(agrup_cep)

print(geo_ref.head())

   geolocation_zip_code_prefix  geolocation_lat  geolocation_lng
0                         1001       -23.550190       -46.634024
1                         1002       -23.548146       -46.634979
2                         1003       -23.548994       -46.635731
3                         1004       -23.549799       -46.634757
4                         1005       -23.549456       -46.636733


In [3]:
df_logistica = pd.read_csv('tabela_logistica.csv')

df_final = df_logistica.merge(geo_ref,
                              left_on = 'cep_vendedor',
                              right_on = 'geolocation_zip_code_prefix',
                              how = 'left')

df_final = df_final.rename(columns = {'geolocation_lat': 'lat_vendedor', 
                                    'geolocation_lng': 'lng_vendedor'})

df_final = df_final.merge(geo_ref,
                              left_on = 'cep_cliente',
                              right_on = 'geolocation_zip_code_prefix',
                              how = 'left')

df_final = df_final.rename(columns = {'geolocation_lat': 'lat_cliente', 
                                    'geolocation_lng': 'lng_cliente'})

print(df_final.columns)

Index(['order_id', 'product_id', 'seller_id', 'data_compra', 'data_aprovacao',
       'data_postagem', 'data_entrega', 'data_estimada', 'valor_frete',
       'cep_cliente', 'uf_cliente', 'cidade_cliente', 'cep_vendedor',
       'uf_vendedor', 'cidade_vendedor', 'geolocation_zip_code_prefix_x',
       'lat_vendedor', 'lng_vendedor', 'geolocation_zip_code_prefix_y',
       'lat_cliente', 'lng_cliente'],
      dtype='object')


In [4]:
import numpy as np

# 1. Definindo a Função de Haversine
def haversine_distance(lat1, lon1, lat2, lon2):
    r=6371
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)

    a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2)**2
    res = r * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))
    return np.round(res, 2)

# 2. Aplicando a função na nossa tabela
df_final['distancia_km'] = haversine_distance(
    df_final['lat_vendedor'],
    df_final['lng_vendedor'],
    df_final['lat_cliente'],
    df_final['lng_cliente']
)

print(df_final[['cep_vendedor', 'cep_cliente', 'distancia_km']].head())

   cep_vendedor  cep_cliente  distancia_km
0         27277        28013        301.50
1          3471        15775        585.56
2         37564        35661        312.34
3         14403        12952        293.17
4         87900        13226        646.16


In [5]:
print(df_final['distancia_km'].describe())

count    109661.000000
mean        596.233797
std         588.541218
min           0.000000
25%         185.210000
50%         431.860000
75%         791.630000
max        8677.910000
Name: distancia_km, dtype: float64


In [6]:
# 1. Ver a correlação matemática (Varia de -1 a 1)
# Estamos comparando a coluna 'distancia_km' com 'valor_frete'
print("--- Correlação da distância ---")
print(df_final[['distancia_km', 'valor_frete']].corr())

--- Correlação da distância ---
              distancia_km  valor_frete
distancia_km      1.000000     0.391966
valor_frete       0.391966     1.000000


In [7]:
df_produtos = pd.read_csv('olist_products_dataset.csv')

# Filtra só o que importa (ID, Peso e Medidas)
df_produtos = df_produtos[['product_id', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']]

# Criar uma coluna de "Volume" (Cúbico)
df_produtos['volume_cm3'] = (df_produtos['product_length_cm'] * df_produtos['product_height_cm'] * df_produtos['product_width_cm'])

# Junta com nossa tabela principal (df_final)
df_completo = df_final.merge(df_produtos, on='product_id', how='left')

# Limpeza rápida (remover produtos sem peso cadastrado)
df_completo = df_completo.dropna(subset=['product_weight_g', 'volume_cm3'])

print(df_completo[['product_id', 'distancia_km', 'product_weight_g', 'volume_cm3']].head())

                         product_id  distancia_km  product_weight_g  \
0  4244733e06e7ecb4970a6e2683c13e61        301.50             650.0   
1  e5f2d52b802189ee658865ca93d83a8f        585.56           30000.0   
2  c777355d18b72b67abbeef9df44fd0fd        312.34            3050.0   
3  7634da152a4610f1595efa32f14722fc        293.17             200.0   
4  ac6c3623068f30de03045865e4e10089        646.16            3750.0   

   volume_cm3  
0      3528.0  
1     60000.0  
2     14157.0  
3      2400.0  
4     42000.0  


In [8]:
colunas_analise = ['valor_frete', 'distancia_km', 'product_weight_g', 'volume_cm3']

print("--- Matriz de Correlação ---")
print(df_completo[colunas_analise].corr())

--- Matriz de Correlação ---
                  valor_frete  distancia_km  product_weight_g  volume_cm3
valor_frete          1.000000      0.391972          0.610279    0.583328
distancia_km         0.391972      1.000000         -0.006698   -0.009906
product_weight_g     0.610279     -0.006698          1.000000    0.805322
volume_cm3           0.583328     -0.009906          0.805322    1.000000


In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import pandas as pd

# Limpeza de dados
# O subset garante que ele só apague se faltar dado nessas colunas específicas
df_limpo = df_completo.dropna(subset=['distancia_km', 'volume_cm3', 'valor_frete'])

print(f"Linhas antes: {len(df_completo)} -> Linhas depois da limpeza: {len(df_limpo)}")

# Seleção de Features (Usando o dataframe limpo)
X = df_limpo[['distancia_km', 'volume_cm3']]
y = df_limpo['valor_frete']

# Separa Treino e Teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treina o Modelo
modelo_final = LinearRegression()
modelo_final.fit(X_train, y_train)

# 5. Avaliar
score = r2_score(y_test, modelo_final.predict(X_test))
print(f"📊 Nota do Modelo (R²): {score:.2f}")

print("-" * 30)
print("🧠 A Fórmula Final:")
print(f"Taxa Fixa.............: R$ {modelo_final.intercept_:.2f}")
print(f"Custo por KM..........: R$ {modelo_final.coef_[0]:.4f}")
print(f"Custo por cm³ (Volume): R$ {modelo_final.coef_[1]:.6f}")

Linhas antes: 110179 -> Linhas depois da limpeza: 109643
📊 Nota do Modelo (R²): 0.48
------------------------------
🧠 A Fórmula Final:
Taxa Fixa.............: R$ 7.50
Custo por KM..........: R$ 0.0107
Custo por cm³ (Volume): R$ 0.000400


In [11]:
from IPython.display import clear_output

print("="*50)
print("🚛 SISTEMA DE CÁLCULO DE FRETES - VERSÃO INTERATIVA 🚛")
print("="*50)

while True:
    print ("\n👇 Nova Simulação (Digite 'sair' para encerrar):")

    entrada_dist = input("📍 Digite a Distância (KM): ")
    if entrada_dist.lower() == "sair":
        print("👋 Encerrando o sistema. Até logo!")
        break

    print(f"📍 Digite a Distância (KM): {entrada_dist}")

    entrada_vol = input("📦 Digite o Volume (cm³): ")

    if entrada_vol.lower() == 'sair':
        print("👋 Encerrando o sistema. Até logo!")
        break

    print(f"📦 Digite o Volume (cm³):  {entrada_vol}")

    try:
        distancia = float(entrada_dist)
        volume = float(entrada_vol)
        
        dados_input = pd.DataFrame({
            'distancia_km': [distancia],
            'volume_cm3': [volume]
        })

        preco_previsto = modelo_final.predict(dados_input)[0]

        print("-" * 30)
        print(f"✅ Custo Estimado do Frete: R$ {preco_previsto:.2f}")
        print("-" * 30)

    except ValueError:
        print("⚠️ ERRO: Por favor, digite apenas números (use ponto para decimais).")
    except Exception as e:
        print(f"⚠️ Erro inesperado: {e}")

🚛 SISTEMA DE CÁLCULO DE FRETES - VERSÃO INTERATIVA 🚛

👇 Nova Simulação (Digite 'sair' para encerrar):
📍 Digite a Distância (KM): 200
📦 Digite o Volume (cm³):  800
------------------------------
✅ Custo Estimado do Frete: R$ 9.96
------------------------------

👇 Nova Simulação (Digite 'sair' para encerrar):
📍 Digite a Distância (KM): 200
📦 Digite o Volume (cm³):  80000
------------------------------
✅ Custo Estimado do Frete: R$ 41.63
------------------------------

👇 Nova Simulação (Digite 'sair' para encerrar):
👋 Encerrando o sistema. Até logo!
